In [2]:
#Program to demonstrate the BB84 protocol in qiskit.
from qiskit import *
import numpy as np

In [3]:
#Alice's function to encode the bit string
#Alice uses the random bit string to select the qubit basis in which the bits are encoded.
#In the {0,1} basis, bit 0 is encoded as ket(0) and bit 1 as ket(1)
#In the {+,-} basis, bit 0 is encoded as ket(+) and bit 1 as ket(-)

def Alice_encode(rand_str_Alice,info_str,qc):
    i=0
    for (bit,info) in zip(rand_str_Alice,info_str):
        if bit=='0': #use the {0,1} basis to encode
            if info=='1':
                qc.x(i)
        else:         #use the {+,-} basis to encode
            qc.h(i)
            if info=='1':
                qc.z(i)
        i+=1
        
    qc.barrier()
    return qc  


In [4]:
#Alice's function to extract the secret key after public knowledge of the choice of basis

def Alice_extract_key(rand_str_Alice,rand_str_Bob,info_str):
    secret_key_Alice=''
    for (bit_a,bit_b,bit_c) in zip(rand_str_Alice,rand_str_Bob,info_str):
        if bit_a==bit_b:  #Keeping only those bits for which the basis was the same
            secret_key_Alice+=bit_c
    return secret_key_Alice                  

In [5]:
#Bob's function to perform the measurement on the qubits received

def Bob_measure(rand_str_Bob,qc):
    j=0
    for bit in rand_str_Bob:
        if bit=='1':  #measure in the {0,1} basis if bit=0, else in the {+,-} basis
            qc.h(j)   #Required for effective measurement in the {+,-} basis
        qc.measure(j,j)
        j+=1
        
    backend_req=Aer.get_backend('qasm_simulator')  #setting up the backend
    result=execute(qc,backend_req,shots=1).result() #need to measure only once
    counts=result.get_counts()  #This contains a dictionary with key=measurement result, value=count of that result
    #print(counts)    #NOTE-qiskit orders the qubits in reverse: q_5,q_4...q_0    
    qc.barrier()
    
    #To extract the bit string for measurement results-
    Bob_results=''
    for key in counts:
        Bob_results+=key
    Bob_results=Bob_results[::-1] #NOTE-qiskit orders the qubits in reverse: q_5,q_4...q_0
    
    return qc,Bob_results        
            

In [6]:
#Bob's function to extract the secret key after public knowledge of the choice of basis

def Bob_extract_key(rand_str_Alice,rand_str_Bob,Bob_results):
    secret_key_Bob=''
    for (bit_a,bit_b,bit_c) in zip(rand_str_Alice,rand_str_Bob,Bob_results):
        if bit_a==bit_b:   #Keeping only those results for which the basis was the same
            secret_key_Bob+=bit_c  
            
    return secret_key_Bob


In [7]:
#Defining the required values
n=6                #length of the information string
rand_str_Alice='011011' #random string to select the basis for Alice, the source of this string can be changed
rand_str_Bob='010001'   #random string to select the basis for Bob
info_str='100110'  #bit string to set up the secret key
qc=QuantumCircuit(n,n) #no. of qubits should be equal to the length of the information string

#Both the random bit string and the information bit string must be of the same length
if len(rand_str_Alice)!=len(info_str):
    print("Length of the random bit string and the information bit string is unequal!")
    #exit()

In [8]:
#Encoding done by Alice

qc=Alice_encode(rand_str_Alice,info_str,qc)
#qc.draw(output='mpl')

#qubits are then transferred to Bob through a quantum channel, here it is assumed that thsi is done.

In [9]:
#Measurement done by Bob using his own random bit string to decide the measurement basis

qc,Bob_results=Bob_measure(rand_str_Bob,qc)
#qc.draw(output='mpl')


In [10]:
#Now Bob and Alice make their choice of basis public. In this demonstration, it is assumed that this is done.
#If the choice of basis is different, the measurement result is discarded, else it forms the secret key

secret_key_Alice=Alice_extract_key(rand_str_Alice,rand_str_Bob,info_str)
secret_key_Bob=Bob_extract_key(rand_str_Alice,rand_str_Bob,Bob_results)

#Both values should be the same for a successful key distribution
print(secret_key_Alice) 
print(secret_key_Bob)
#This secret key can then be used for communicating messages between Alice and Bob.

1010
1010
